# Pseudopotentials

It was discussed in the lecture that plane-wave DFT codes usually employ so-called pseudopotentials (PSP) to model the interaction of electrons and nuclei. Replacing the true Coulombic interaction by a PSP is an approximation. Multiple types of PSPs exist and from a mathematical perspective little is understood about the errors introduced by PSPs. However, from a physical point of view using PSPs is reasonable as it only effects the electron density close to the nuclei, which for most phaenomena only plays a minor role. On the upside PSPs turn out to make plane-wave calculations much more feasible.

This aspect we will investigate numerically in this exercise. Our goal will be to determine the energy of a single neon atom using the so-called LDA approximation up to an absolute error of `0.1`. We will do this by an explicit convergence study, i.e. by increasing the basis size and improving the numerics until we are sure we have found the energy to this tolerance.

Our computational setup is simple: We will put a single neon atom into a cubic box of length $a$. As DFTK uses periodic boundary conditions, this atom interacts spuriously with the neighbouring cells, introducing an error to our calculation. As $a \to \infty$ this error disappears, thus in principle convergence with increasing $a$ should be one parameter to study. For simplicitity (as large values of $a$ lead to very costly calculations) we will ignore this aspect here.

The convergence parameter we will not ignore, however, is the $E_\text{cut}$ parameter discussed in the lecture. This parameter determines the basis set size, thus the accuracy of the calculation. Again calculations get better for $E_\text{cut} \to \infty$.

In the language of DFTK the calculation we want to perform is:

In [ ]:
using LinearAlgebra
using DFTK

# Numerical parameters
#
Ecut = 20  # Plane-wave basis cutoff

# Use a HGH pseudopotential:
Ne = ElementPsp(:Ne, psp=load_psp("hgh/lda/ne-q8"))

# Use the true Coulomb interaction:
# Ne = ElementCoulomb(:Ne)
#
# End numerical parameters 

# Setup Cubic box and place neon atom in the middle
a = 10    # Box size 
lattice   = a * Matrix(I, 3, 3)
atoms     = [Ne]
positions = [[0.5, 0.5, 0.5]]

# Use LDA model, discretise and run SCF algorithm
model  = model_LDA(lattice, atoms, positions)
basis  = PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])
scfres = self_consistent_field(basis)
Etot   = scfres.energies.total

# Print total energy
println("\nTotal DFT energy is $Etot")

**Warning:** DFT calculations can be both time and memory consuming. When doing these convergence studies therefore start with small values for `Ecut` (like the ones shown here) and increase slowly, especially if you are running these on a laptop with 8GB of main memory or less. You are not expected to re-compute the provided reference results.

**Exercise:**

(a) First stick with the pseudpotential (PSP) version of the neon atom (`ElementPsp`). Converge the energy to an absolute error of `0.1`. The easiest way to do this is to run multiple calculations for different values of `Ecut` and to plot the error against a reference on a semilog scale. A good reference is at `Ecut = 600`, where the DFT energy is `-34.85376`.<br />
*Hint:* If you want to do the plotting within Julia good, take a look at the [Plots.jl](http://docs.juliaplots.org/latest/gallery/gr/) documentation for some nice examples. Another good plotting package could be [PyPlot.jl](https://github.com/JuliaPy/PyPlot.jl), which has exactly the same interface as the `matplotlib` Python package.

(b) Repeat the exercise for the all-electron case (`ElementCoulomb`). Here the reference result at `Ecut = 600` is `-123.5983`. Explore the convergence all the way up to `Ecut=100`. What differences in the obtained total energy as well as the convergence with `Ecut` do you observe? Suggest an explanaition keeping in mind that part of the idea of the PSP is to avoid the explicit treatment of the core electrons.